# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("Data Wrangling").getOrCreate()

In [3]:
user_log = spark.read.json('data/sparkify_log_small.json')

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
# TODO: write your code to answer question 1

pages = user_log.select("page").dropDuplicates().collect()
pages = set(pages)

user_pages = user_log.select("page").where(user_log.userId == "").dropDuplicates().collect()
user_pages = set(user_pages)

print('The user empty did not visit the following pages: ')
for item in list(pages - user_pages):
    print(item["page"])

The user empty did not visit the following pages: 
Upgrade
Submit Downgrade
Settings
Logout
Submit Upgrade
Downgrade
NextSong
Save Settings
Error


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [5]:
# TODO: use this space to explore the behavior of the user with an empty string

user_pages = user_log.select("page").where(user_log.userId == "").dropDuplicates().show()
user_pages

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



**These pages does not requires the user to be registered nor logged in. Hence this could be the user_id for all users who have not logged in to the app.**

# Question 3

How many female users do we have in the data set?

In [6]:
# TODO: write your code to answer question 3

female_users = user_log.select(["userId", "gender"]).dropDuplicates().groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|  null|    1|
|     M|  501|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [7]:
# TODO: write your code to answer question 4

song_play = user_log.select(["artist", "song"]).where(user_log.page == "NextSong")
most_palyed_artist = song_play.groupby("artist").count().orderBy("count", ascending=False).show()
most_palyed_artist

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
|Red Hot Chili Pep...|   24|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [8]:
# TODO: write your code to answer question 5

from pyspark.sql.functions import udf, desc, col
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window.partitionBy('userID').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')).select('userID', 'page', 'ts')\
    .withColumn('homevisit', function(col('page'))).withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')).groupBy('userID', 'period') \
    .agg({'period':'count'}).agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

